## Step 1: Import packages and data

In [1]:
%matplotlib notebook
# %matplotlib ipympl
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
import itertools as it
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from hmmlearn import hmm
from hmmlearn import base


import warnings
warnings.filterwarnings('ignore')
desired_width=320
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns',20)
np.set_printoptions(linewidth=desired_width)
np.set_printoptions(threshold=sys.maxsize)

#### Generate Dataframe of all keypoints from JSON file

In [2]:
keypoints_all = pd.read_json('./Keypoints_All.json', orient='records')
new_columns = ['Gesture', 'Sub folder No.', 'Frame No.', 'Joint', 'X', 'Y', 'Probability', 'Depth']
keypoints_all = keypoints_all.reindex(columns = new_columns)
print('keypoints_all shape', keypoints_all.shape)
keypoints_all.head()

keypoints_all shape (181275, 8)


,Gesture,Sub folder No.,Frame No.,Joint,X,Y,Probability,Depth
0,1,1,1,0,588.192078,143.525146,0.911283,0.619608
1,1,1,1,1,588.338928,210.050186,0.897833,0.639216
2,1,1,1,2,543.214905,210.081268,0.856357,0.639216
3,1,1,1,3,525.464661,284.558472,0.859284,0.647059
4,1,1,1,4,521.571350,351.166412,0.869531,0.639216


#### Generate Dataframe of keypoints 1-7

In [3]:
keypoints_1_7 = keypoints_all[keypoints_all['Joint'].isin(range(1,8))]
print('keypoints_1_7 shape', keypoints_1_7.shape)

keypoints_1_8 shape (50757, 8)


## Step 2: Filter out bad recordings (using OpenPose probability)

#### Check Probability (from OpenPose output) per folder

In [4]:
fig, axes = plt.subplots(2,1, sharex=True)
pd.pivot_table(keypoints_1_7, index='Gesture', columns = 'Sub folder No.', values='Probability', aggfunc='mean').plot(kind='bar', ax=axes[0], title='Average of all keypoints\' Probability per folder', legend=True, alpha=0.5, yticks=np.arange(0, 1.2, 0.2))
pd.pivot_table(keypoints_1_7, index='Gesture', columns = 'Sub folder No.', values='Probability', aggfunc='var').plot(kind='bar', ax=axes[1], title='Variance of all keypoints\' Probability per folder', legend=True, alpha=0.5, yticks=np.arange(0, 0.012, 0.002))

<IPython.core.display.Javascript object>

#### Observe distribution of Probability

In [5]:
fig = plt.figure()
keypoints_1_7['Probability'].plot.hist(bins=100, title='Distribution of \"Probability\" (OpenPose output)', alpha=0.5)

<IPython.core.display.Javascript object>

#### Get Probability value at percentile 1, use it as probability threshold for later data cleaning

In [6]:
prob_threshold = keypoints_1_7['Probability'].quantile(0.01)
print(prob_threshold)

0.72871965172


#### Calculate the percentage of "low probability keypoints" (keypoints probability < probability threshold) per foler

In [7]:
keypoints_1_7_folder_prob_threshold_percent = keypoints_1_7.pivot_table(index = ['Gesture', 'Sub folder No.'], values='Probability', aggfunc = lambda x:np.count_nonzero(x<prob_threshold)/len(x)*100)
keypoints_1_7.pivot_table(index = 'Gesture', columns = 'Sub folder No.', values='Probability', aggfunc = lambda x:np.count_nonzero(x<prob_threshold)/len(x)*100).plot.bar(title='Before folder filter, Percentage of \"low probability keypoints\" per folder', legend=True, alpha=0.5, yticks=np.arange(0, 13, 2))

<IPython.core.display.Javascript object>

#### Filter the folders and keey only the folders has less than 8 percent of "low probability keypoints" 

In [8]:
low_prob_keypoints_percentage_threshold = 8
keypoints_1_7_folder_filtered_index = keypoints_1_7_folder_prob_threshold_percent[keypoints_1_7_folder_prob_threshold_percent['Probability'] < low_prob_keypoints_percentage_threshold].index.tolist()
keypoints_1_7_folder_filtered = keypoints_1_7[keypoints_1_7[['Gesture','Sub folder No.']].apply(lambda x:(x[0],x[1]) in keypoints_1_7_folder_filtered_index, axis=1)]

print('keypoints_1_7_filtered shape', keypoints_1_7_folder_filtered.shape)

keypoints_1_8_filtered shape (50757, 8)


In [9]:
keypoints_1_7_folder_filtered.pivot_table(index = 'Gesture', columns = 'Sub folder No.', values='Probability', aggfunc = lambda x:np.count_nonzero(x<prob_threshold)/len(x)*100).plot.bar(title='After folder filter, Percentage of \"low probability keypoint\" per folder', legend=True, alpha=0.5, yticks=np.arange(0, 13, 2))

<IPython.core.display.Javascript object>

In [10]:
fig, axes = plt.subplots(2,1, sharex=True)
pd.pivot_table(keypoints_1_7_folder_filtered, index='Gesture', columns = 'Sub folder No.', values='Probability', aggfunc='mean').plot(kind='bar', ax=axes[0], title='Average of all keypoints\' Probability per folder', legend=True, alpha=0.5, yticks=np.arange(0, 1.2, 0.2))
pd.pivot_table(keypoints_1_7_folder_filtered, index='Gesture', columns = 'Sub folder No.', values='Probability', aggfunc='var').plot(kind='bar', ax=axes[1], title='Variance of all keypoints\' Probability per folder', legend=True, alpha=0.5, yticks=np.arange(0, 0.012, 0.002))

<IPython.core.display.Javascript object>

## Step 3: Clean frames (using Weighted Moving Average Smoothing)

### 1. Clean OpenPose outputs (X, Y)

#### Check the X values of Joint 3 in all frames, we can see there are 0 values, this also need to be cleaned

In [11]:
# keypoints_1_7_folder_filtered[keypoints_1_7_folder_filtered['Joint']==3].pivot_table(index = 'Frame No.', columns = ['Gesture', 'Sub folder No.'], values='X').plot.line(alpha=0.5)

#### We use Weighted Moving Average Smoothing to clean the 0 values of X and Y.
#### First, prepare a dataframe with t-1 and t+1 values for each keypoints

In [12]:
# df = pd.concat([keypoints_1_7_folder_filtered, keypoints_1_7_folder_filtered.shift(7)[['X', 'Y', 'Probability']].rename(columns={'X':'X(t-1)', 'Y':'Y(t-1)', 'Probability':'Probability(t-1)'}), keypoints_1_7_folder_filtered.shift(-7)[['X', 'Y', 'Probability']].rename(columns={'X':'X(t+1)', 'Y':'Y(t+1)', 'Probability':'Probability(t+1)'})], axis=1)
# df.head(14)

#### Use Weighted Moving Average Smoothing to calculate X_cleaned and Y_cleaned

In [13]:
# X_cleaned = df.apply(lambda row: (row['X']*row['Probability'] + row['X(t-1)']*row['Probability(t-1)'] + row['X(t+1)']*row['Probability(t+1)'])/(row['Probability']+row['Probability(t-1)']+row['Probability(t+1)']) if row['Probability']==0 else row['X'], axis=1)
# Y_cleaned = df.apply(lambda row: (row['Y']*row['Probability'] + row['Y(t-1)']*row['Probability(t-1)'] + row['Y(t+1)']*row['Probability(t+1)'])/(row['Probability']+row['Probability(t-1)']+row['Probability(t+1)']) if row['Probability']==0 else row['Y'], axis=1)

#### Add cleaned X and Y to the original dataframe

In [14]:
# keypoints_1_7_frame_cleaned = pd.concat([keypoints_1_7_folder_filtered, X_cleaned.rename('X_cleaned'), Y_cleaned.rename('Y_cleaned')], axis=1)
# keypoints_1_7_frame_cleaned[keypoints_1_7_frame_cleaned['Probability']==0].head()

#### Recheck the X values of Joint 3 in all frames, we can see no more 0 values

In [15]:
# keypoints_1_7_frame_cleaned[keypoints_1_7_frame_cleaned['Joint']==3].pivot_table(index = 'Frame No.', columns = ['Gesture', 'Sub folder No.'], values='X_cleaned').plot.line(alpha=0.5)

#### After data cleaning of OpenPose data, we also need to clean depth data

#### Check the depth value when probability == 0 (aka. X==0 and Y==0)

In [16]:
# keypoints_1_7_frame_cleaned[keypoints_1_7_frame_cleaned['Probability']==0].pivot_table(index = ['Frame No.', 'Joint'], columns = ['Gesture', 'Sub folder No.'], values='Depth').swapaxes(axis1=0, axis2=1)

#### When Probability == 0, it means that the keypoint is occluded. To estimate the depth value, we use Moving Average Smoothing: get average of depth data at t-1 and t+1

In [17]:
# df = pd.concat([keypoints_1_7_frame_cleaned, keypoints_1_7_frame_cleaned.shift(7)['Depth'].rename('Depth(t-1)'), keypoints_1_7_frame_cleaned.shift(-7)['Depth'].rename('Depth(t+1)')], axis=1)
# Depth_cleaned = df[['Probability', 'Depth', 'Depth(t-1)', 'Depth(t+1)']].apply(lambda x: np.mean((x[2], x[3])) if x[0]==0 else x[1], axis=1)
# keypoints_1_7_frame_cleaned['Depth_cleaned'] = Depth_cleaned
# keypoints_1_7_frame_cleaned[keypoints_1_7_frame_cleaned['Probability']==0].head()

### 2. Clean depth from RealSense camera

#### Now check the depth data of one joint (Joint 4 for example) of all gestures, we can see abnormal depth values (noise)

In [18]:
keypoints_1_7_folder_filtered[np.logical_and(keypoints_1_7_folder_filtered['Joint']==4, keypoints_1_7_folder_filtered['Gesture']<8)].pivot_table(index = 'Frame No.', columns = ['Gesture', 'Sub folder No.'], values='Depth').plot.line(alpha=0.5, legend=False, yticks=np.arange(0, 1.1, 0.1))

<IPython.core.display.Javascript object>

#### Calculate the depth median per Joint of all frames in each Sub folder of each Gesture

In [19]:
Depth_median = keypoints_1_7_folder_filtered.pivot_table(index = ['Gesture', 'Sub folder No.', 'Joint'], values = 'Depth', aggfunc = lambda x: np.median(x)).rename(columns={'Depth':'Depth_median'})
Depth_median.head()

Depth_median
Gesture Sub folder No. Joint              
1       1              1          0.639216
                       2          0.639216
                       3          0.639216
                       4          0.631373
                       5          0.639216

#### Join 'Depth median' into previous keypoint dataframe, then calculate 'Depth - Depth median'  for each depth value

In [20]:
# Join depth median into previous keypoint dataframe
df = keypoints_1_7_folder_filtered.join(Depth_median, on=['Gesture', 'Sub folder No.', 'Joint'])
# Calculate the gap between depth and depth median for each depth value
df['Depth-Depth_median'] = df['Depth'] - df['Depth_median']
df.head()

,Gesture,Sub folder No.,Frame No.,Joint,X,Y,Probability,Depth,Depth_median,Depth-Depth_median
1,1,1,1,1,588.338928,210.050186,0.897833,0.639216,0.639216,0.000000
2,1,1,1,2,543.214905,210.081268,0.856357,0.639216,0.639216,0.000000
3,1,1,1,3,525.464661,284.558472,0.859284,0.647059,0.639216,0.007843
4,1,1,1,4,521.571350,351.166412,0.869531,0.639216,0.631373,0.007843
5,1,1,1,5,635.311523,210.016373,0.860882,0.639216,0.639216,0.000000


#### Plot the distribution of 'Depth - Depth median' of all depth values

In [21]:
# distribution = df.pivot_table(index = ['Sub folder No.', 'Frame No.'], columns = ['Gesture', 'Joint'], values = 'Depth-Depth_median')
distribution = df.pivot_table(index = ['Gesture', 'Sub folder No.', 'Frame No.', 'Joint'], values = 'Depth-Depth_median')
display(distribution.head())
distribution.plot.hist(bins=100, alpha=0.5, legend=False)

Depth-Depth_median
Gesture Sub folder No. Frame No. Joint                    
1       1              1         1                0.000000
                                 2                0.000000
                                 3                0.007843
                                 4                0.007843
                                 5                0.000000

<IPython.core.display.Javascript object>

#### Calculate 'Depth - Depth median' at percentile 0.5 and 99.5, use it as confidence interval for later depth cleaning 

In [22]:
# depth_percentile_left = df.pivot_table(index = ['Gesture', 'Joint'], values = 'Depth-Depth_median', aggfunc = lambda x: x.quantile(0.02)).rename(columns={'Depth - Depth_median':'Depth_confidence_interval_left'})
# depth_percentile_right = df.pivot_table(index = ['Gesture', 'Joint'], values = 'Depth-Depth_median', aggfunc = lambda x: x.quantile(0.98)).rename(columns={'Depth - Depth_median':'Depth_confidence_interval_right'})
# display(depth_percentile_left.head())
# display(depth_percentile_right.head())
confidence_interval_left = distribution['Depth-Depth_median'].quantile(0.003)
confidence_interval_right = distribution['Depth-Depth_median'].quantile(0.997)
print('confidence_interval_left =', confidence_interval_left)
print('confidence_interval_right =', confidence_interval_right)

confidence_interval_left = -0.06274509800000005
confidence_interval_right = 0.08823529414999998


#### Calculate the number of "depth out of confidence interval" per Joint

In [23]:
df.pivot_table(index = ['Joint'], values = 'Depth-Depth_median', aggfunc = lambda x: np.count_nonzero(np.logical_or(x<confidence_interval_left, x>confidence_interval_right))).plot.bar(title='Before depth clean, number of \"depth out of confidence interval\" per Joint', legend=False, alpha=0.5, yticks=np.arange(0, 400, 50))

<IPython.core.display.Javascript object>

#### Calculate the percentage of "depth out of confidence interval" per Gesture and Sub folder

In [24]:
df.pivot_table(index = ['Gesture', 'Sub folder No.'], columns = ['Joint'], values = 'Depth-Depth_median', aggfunc = lambda x: np.count_nonzero(np.logical_or(x<confidence_interval_left, x>confidence_interval_right))/len(x)*100).plot.bar(title='Before depth clean, percentage of \"depth out of confidence interval\" per Gesture and Sub folder', legend=True, alpha=0.5, yticks=np.arange(0, 45, 5))

<IPython.core.display.Javascript object>

#### We use Moving Average Smoothing to clean the depth values out of confidence interval.
#### First, prepare a dataframe with t-1 and t+1 values for each keypoints:

In [25]:
# df2 = pd.concat([df, df.shift(7)[['Depth', 'Depth-Depth_median']].rename(columns={'Depth':'Depth (t-1)', 'Depth - Depth_median':'Depth-Depth_median (t-1)'}), df.shift(-7)[['Depth', 'Depth-Depth_median']].rename(columns={'Depth':'Depth (t+1)', 'Depth - Depth_median':'Depth-Depth_median (t+1)'})], axis=1)
df['Depth (t-1)'] = df['Depth'].shift(7)
df['Depth (t+1)'] = df['Depth'].shift(-7)
df.head(30000)

,Gesture,Sub folder No.,Frame No.,Joint,X,Y,Probability,Depth,Depth_median,Depth-Depth_median,Depth (t-1),Depth (t+1)
1,1,1,1,1,588.338928,210.050186,0.897833,0.639216,0.639216,0.000000,NaN,0.639216
2,1,1,1,2,543.214905,210.081268,0.856357,0.639216,0.639216,0.000000,NaN,0.639216
3,1,1,1,3,525.464661,284.558472,0.859284,0.647059,0.639216,0.007843,NaN,0.658824
4,1,1,1,4,521.571350,351.166412,0.869531,0.639216,0.631373,0.007843,NaN,0.639216
5,1,1,1,5,635.311523,210.016373,0.860882,0.639216,0.639216,0.000000,NaN,0.647059
...,...,...,...,...,...,...,...,...,...,...,...,...
107126,5,12,37,1,588.203430,196.329254,0.931189,0.631373,0.631373,0.000000,0.631373,0.631373
107127,5,12,37,2,539.332031,196.431213,0.893481,0.639216,0.631373,0.007843,0.639216,0.639216
107128,5,12,37,3,521.582275,274.771179,0.827447,0.611765,0.631373,-0.019608,0.631373,0.603922
107129,5,12,37,4,494.281616,270.798981,0.839903,0.556863,0.603922,-0.047059,0.572549,0.647059


#### Create a function to do Moving Average Smoothing for depth values out of confidence interval:

In [26]:
def depth_clean(df, times=1):
    for _ in range(times):
        depth_cleaned = df.apply(lambda row: (row['Depth (t-1)'] + row['Depth (t+1)'])/2 if (row['Depth-Depth_median']<confidence_interval_left or row['Depth-Depth_median']>confidence_interval_right) else row['Depth'], axis=1)
#         depth_cleaned = df.apply(lambda row: (row['Depth (t-1)'] + row['Depth'] + row['Depth (t+1)'])/3 if (row['Depth-Depth_median']<confidence_interval_left or row['Depth-Depth_median']>confidence_interval_right) else row['Depth'], axis=1)
        df['Depth'] = depth_cleaned
        df['Depth-Depth_median'] = depth_cleaned - df['Depth_median']
        df['Depth (t-1)'] = depth_cleaned.shift(7)
        df['Depth (t+1)'] = depth_cleaned.shift(-7)
    return df

#### Perform Moving Average Smoothing for 20 times:

In [27]:
keypoints_1_7_depth_cleaned = depth_clean(df.copy(), times=5)

#### After depth clean, recheck the number of "depth out of confidence interval" per Joint

In [28]:
keypoints_1_7_depth_cleaned.pivot_table(index = ['Joint'], values = 'Depth-Depth_median', aggfunc = lambda x: np.count_nonzero(np.logical_or(x<confidence_interval_left, x>confidence_interval_right))).plot.bar(title='After depth clean, number of \"depth out of confidence interval\" per Joint', legend=False, alpha=0.5, yticks=np.arange(0, 400, 50))

<IPython.core.display.Javascript object>

#### After depth clean, recheck the percentage of "depth out of confidence interval" per Gesture and Sub folder

In [29]:
keypoints_1_7_depth_cleaned.pivot_table(index = ['Gesture', 'Sub folder No.'], columns = ['Joint'], values = 'Depth-Depth_median', aggfunc = lambda x: np.count_nonzero(np.logical_or(x<confidence_interval_left, x>confidence_interval_right))/len(x)*100).plot.bar(title='After depth clean, number of \"depth out of confidence interval\" per Gesture and Sub folder', legend=True, alpha=0.5, yticks=np.arange(0, 45, 5))

<IPython.core.display.Javascript object>

#### After depth clean, recheck the depth data of one joint (Joint 4 for example) of all gestures, we can see the depth values are much smoother, abnormal values (noises) have been cleaned

In [30]:
keypoints_1_7_depth_cleaned[np.logical_and(keypoints_1_7_depth_cleaned['Joint']==4, keypoints_1_7_depth_cleaned['Gesture']<8)].pivot_table(index = 'Frame No.', columns = ['Gesture', 'Sub folder No.'], values='Depth').plot.line(alpha=0.5, legend=False, yticks=np.arange(0, 1.1, 0.1))

<IPython.core.display.Javascript object>

#### Keep the cleaned depth, and remove the columns used for depth cleaning

In [31]:
keypoints_1_7_cleaned = keypoints_1_7_depth_cleaned.copy()
new_columns = ['Gesture', 'Sub folder No.', 'Frame No.', 'Joint', 'X', 'Y', 'Depth']
keypoints_1_7_cleaned = keypoints_1_7_cleaned.reindex(columns = new_columns)
keypoints_1_7_cleaned.describe()

,Gesture,Sub folder No.,Frame No.,Joint,X,Y,Depth
count,50757.000000,50757.000000,50757.000000,50757.00000,50757.000000,50757.000000,50757.000000
mean,4.384912,9.650945,26.452076,4.00000,457.402523,221.201090,0.598103
std,2.314992,5.199421,15.871076,2.00002,141.534747,59.239783,0.049443
min,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,0.313971
25%,2.000000,5.000000,13.000000,2.00000,339.788361,179.303177,0.556863
50%,4.000000,10.000000,26.000000,4.00000,402.494415,208.200470,0.603922
75%,6.000000,14.000000,38.000000,6.00000,588.315063,270.659363,0.639216
max,8.000000,18.000000,78.000000,7.00000,805.754700,364.834869,0.745098


## Step 4: Data normalization

In [32]:
# keypoints_1_7_normalized = keypoints_1_7_cleaned.copy()
# keypoints_1_7_normalized['X'] = keypoints_1_7_cleaned['X']/1280
# keypoints_1_7_normalized['Y'] = keypoints_1_7_cleaned['Y']/720
# keypoints_1_7_normalized.head()

NameError: name 'keypoints_1_7_cleaned' is not defined

In [33]:
keypoints_1_7_normalized = keypoints_1_7_cleaned.copy()
#keypoints_1_7_normalized[np.logical_or(keypoints_1_7_normalized['Sub folder No.'] < 7, keypoints_1_7_normalized['Sub folder No.'].between(10, 12))].replace(['X'], ['X']/1280)
#keypoints_1_7_normalized['X'] = keypoints_1_7_normalized.apply(lambda x: x/1280 if np.logical_or(keypoints_1_7_normalized['Sub folder No.'] < 7, keypoints_1_7_normalized['Sub folder No.'].between(10, 12)))

def FolderselectX(vec):
    Folder = vec[0]
    X = vec[1]
    if Folder <= 6:
        matchVar = X / 1280
    elif 6 < Folder <= 9:
        matchVar = X / 640
    elif 9 < Folder <= 12:
        matchVar = X / 1280
    else:
        matchVar = X / 640
    return matchVar

keypoints_1_7_normalized['X'] = keypoints_1_7_normalized[['Sub folder No.','X']].apply(FolderselectX,axis=1)

def FolderselectY(vec):
    Folder = vec[0]
    X = vec[1]
    if Folder <= 6:
        matchVar = X / 720
    elif 6 < Folder <= 9:
        matchVar = X / 480
    elif 9 < Folder <= 12:
        matchVar = X / 720
    else:
        matchVar = X / 480
    return matchVar

keypoints_1_7_normalized['Y'] = keypoints_1_7_normalized[['Sub folder No.','Y']].apply(FolderselectY,axis=1)

In [34]:
keypoints_1_7_normalized

,Gesture,Sub folder No.,Frame No.,Joint,X,Y,Depth
1,1,1,1,1,0.459640,0.291736,0.639216
2,1,1,1,2,0.424387,0.291780,0.639216
3,1,1,1,3,0.410519,0.395220,0.647059
4,1,1,1,4,0.407478,0.487731,0.639216
5,1,1,1,5,0.496337,0.291689,0.639216
...,...,...,...,...,...,...,...
181253,8,18,43,3,0.465784,0.484994,0.564706
181254,8,18,43,4,0.467809,0.577509,0.564706
181255,8,18,43,5,0.590185,0.368109,0.556863
181256,8,18,43,6,0.612600,0.471525,0.603922


In [35]:
keypoints_1_7_normalized.to_json('keypoints_1_7_normalized_New.json', orient='records')